https://www.kobis.or.kr/kobis/business/main/main.do

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from datetime import datetime, timedelta

In [2]:
best_movie = pd.read_excel("[KOBIS] 역대_박스오피스(2023.05.11).xlsx", header=4, engine="openpyxl")
best_movie['영화명']

0               명량
1             극한직업
2        신과함께-죄와 벌
3             국제시장
4       어벤져스: 엔드게임
          ...     
195    해리 포터와 불의 잔
196         비긴 어게인
197             태풍
198             킹콩
199         부러진 화살
Name: 영화명, Length: 200, dtype: object

In [241]:
movie_name = best_movie['영화명'][0]
# movie_name = '스즈메의 문단속'
keyword = "영화 "+ str(movie_name) +" 평점"
keyword

'영화 명량 평점'

In [284]:
# 셀레니움으로 네이버 창 열기
driver = webdriver.Chrome()
time.sleep(2)
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&oquery=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%EC%A0%95%EB%B3%B4&tqi=iaz3Ysp0YiRssLJ4UA4ssssss8Z-510880"
driver.get(url)
time.sleep(3)

In [277]:
# movie_name = '부러진 화살'
movie_name = '극한직업'
keyword = "영화 "+ str(movie_name) +" 평점"
keyword

'영화 극한직업 평점'

In [278]:
# 검색어 입력 및 검색
driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').clear()
driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').send_keys(keyword)
driver.find_element(By.CSS_SELECTOR,'button.bt_search').click()
time.sleep(2)

In [209]:
# 공감순 선택
driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
time.sleep(1)

In [201]:
# 리뷰 스크롤 내리기
driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()

# 10번
for i in range(0,3):
    driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
    time.sleep(0.5)

In [279]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [180]:
제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
제목

'부러진 화살'

In [181]:
전체 = soup.select_one('span.area_star_number').text
참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
남자 = soup.select_one('div.area_card_male span.area_star_number').text
여자 = soup.select_one('div.area_card_female span.area_star_number').text
print(전체, 참여자수, 남자, 여자)

8.91 7,378 8.68 9.30


In [182]:
_10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
_20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
_30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
_40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
_50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text
print(_10대, _20대, _30대, _40대, _50대)

9.18 8.77 8.88 9.01 8.55


In [280]:
lis = soup.select('div.lego_review_list._scroller ul._item_wrapper li')
li= lis[0]
관람객별점 = li.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
관람객리뷰 = li.select_one('div.area_text_expand span.desc._text').text.strip()
print(관람객별점, 관람객리뷰)

10 가볍게 생각 없이 즐길수 있는 영화 잘봤습니다


In [196]:
# 평론가 클릭 및 스크롤 내리기
driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()

for i in range(0,3):
    driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
    time.sleep(0.5)

In [282]:
lis = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')
li= lis[4]
평론가별점 = li.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
평론가리뷰 = li.select_one('div.area_text_expand').text.strip()
print(평론가별점, 평론가리뷰)

7.0 극한직업으로 치자면 관객의 입맛을 사로잡아야 하는 영화감독도 포함해야 한다. 청춘 코미디 <스물>(2014), 성인 코미디 <바람 바람 바람>(2017)에 이어 경찰 코미디를 내놓은 이병헌 감독은 기본에 충실한 조리법으로 정통 오락 영화의 깔끔한 맛을 되살려낸다. 소재, 캐릭터, 대사, 음악 모든 요소가 웃음을 위해 복무하고 소기의 성과를 거둔다. 농도가 옅지만 페이소스도 스며있다. 직업정신 투철한 배우들의 연기는 수훈감이다. 모처럼 코미디 장르에서 적성을 발휘한 류승룡과 개그 콤비를 이룬 신하균과 오정세의 연기가 입에 잘 맞는 익숙한 맛이라면, 양념 같은 배역을 단숨에 승격시키는 진선규와 공명의 활약이 전에 없던 감칠맛을 낸다. 이 정도 상차림이라면 온 가족이 보는 명절 영화에 적격이다. 시리즈 제작까지 기대해봄 직하다. 펼쳐보기


In [74]:
# 기본정보 클릭 러닝타임 추출
driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
time.sleep(2)

divs = driver.find_elements(By.CSS_SELECTOR,'div.cm_content_area._cm_content_area_info div.detail_info dl div')

for div in divs:
    if '러닝타임' in div.text:
        러닝타임 = div.text.split()[1]
    elif '개봉' in div.text:
        개봉 = datetime.strptime(div.text.split()[1], '%Y.%m.%d.')
print(러닝타임, 개봉)

128분 2014-07-30 00:00:00


In [287]:
# best_movie = pd.read_excel("[KOBIS] 역대_박스오피스(2023.05.11).xlsx", header=4, engine="openpyxl")

# df1 = []
# df2 = []
# df3 = []

# # 셀레니움으로 네이버 창 열기
# driver = webdriver.Chrome()
# url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&oquery=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%EC%A0%95%EB%B3%B4&tqi=iaz3Ysp0YiRssLJ4UA4ssssss8Z-510880"
# driver.get(url)
# time.sleep(1)

# for movie_name in best_movie['영화명']:
    
#     try: 
#         # 검색어 설정
#         keyword = "영화 "+ str(movie_name) +" 평점"

#         # 검색어 입력 및 검색
#         driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').clear()
#         driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').send_keys(keyword)
#         driver.find_element(By.CSS_SELECTOR,'button.bt_search').click()
#         time.sleep(3)

#         # 리뷰 공감순 선택
#         driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
#         time.sleep(2)
#         driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
#         time.sleep(2)


#         # 관람객 리뷰 스크롤 내리기
#         driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
#         time.sleep(1)

#         for i in range(0,2):
#             driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
#             time.sleep(1)

#         # 평론가 클릭 및 스크롤 내리기
#         driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
#         driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()
#         time.sleep(1)

#         for i in range(0,2):
#             driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
#             time.sleep(1)

#         soup = BeautifulSoup(driver.page_source, "html.parser")

#         제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
#         전체 = soup.select_one('span.area_star_number').text
#         참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
#         남자 = soup.select_one('div.area_card_male span.area_star_number').text
#         여자 = soup.select_one('div.area_card_female span.area_star_number').text

#         _10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
#         _20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
#         _30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
#         _40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
#         _50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text

#         # 관람객 리뷰
#         lis1 = soup.select('div.lego_review_list._scroller ul._item_wrapper li')

#         for li1 in lis1:
#             관람객별점 = li1.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
#             관람객리뷰 = li1.select_one('div.area_text_expand span.desc._text').text.strip()
#             df2.append([movie_name, 제목, 관람객별점, 관람객리뷰])

#         # 평론가 리뷰
#         lis2 = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')

#         for li2 in lis2:
#             평론가별점 = li2.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
#             평론가리뷰 = li2.select_one('div.area_text_expand').text.strip()
#             df3.append([movie_name, 제목, 평론가별점, 평론가리뷰])

#         # 기본정보 클릭 러닝타임 추출
#         driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
#         time.sleep(2)

   
#         sub_soup = BeautifulSoup(driver.page_source, "html.parser")
    
#         divs = sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')
        
#         러닝타임 = '-'

#         for div in divs:
#             if '러닝타임' in div.text:
#                 러닝타임 = div.text.split()[1]
#             elif '개봉' in div.text:
#                 개봉 = div.text.split()[1] 

#         df1.append([movie_name, 제목, 러닝타임, 개봉, 전체, 참여자수, 남자, 여자, _10대, _20대, _30대, _40대, _50대])   
    
#     except:
#         print(movie_name)
        
# star = pd.DataFrame(df1, columns=['movie_name', '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
# review = pd.DataFrame(df2, columns=['movie_name', '제목', '관람객별점', '관람객리뷰'])
# critic_review = pd.DataFrame(df3, columns=['movie_name', '제목', '평론가별점', '평론가리뷰'])

In [48]:
# # 검색어 설정
# movie_name = "1987"

In [49]:
# keyword = "영화 "+ str(movie_name) +" 평점"

# # 검색어 입력 및 검색
# driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').clear()
# driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').send_keys(keyword)
# driver.find_element(By.CSS_SELECTOR,'button.bt_search').click()
# time.sleep(2)

# # 리뷰 공감순 선택
# driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
# time.sleep(1)
# driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
# time.sleep(1)


# # 관람객 리뷰 스크롤 내리기
# driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
# time.sleep(1)

# for i in range(0,3):
#     driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
#     time.sleep(1)

# # 평론가 클릭 및 스크롤 내리기
# driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
# driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()
# time.sleep(1)

# for i in range(0,3):
#     driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
#     time.sleep(1)

# soup = BeautifulSoup(driver.page_source, "html.parser")

# 제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
# 전체 = soup.select_one('span.area_star_number').text
# 참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
# 남자 = soup.select_one('div.area_card_male span.area_star_number').text
# 여자 = soup.select_one('div.area_card_female span.area_star_number').text

# _10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
# _20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
# _30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
# _40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
# _50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text

# # 관람객 리뷰
# lis1 = soup.select('div.lego_review_list._scroller ul._item_wrapper li')

# for li1 in lis1:
#     관람객별점 = li1.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
#     관람객리뷰 = li1.select_one('div.area_text_expand span.desc._text').text.strip()
#     df2.append([movie_name, 제목, 관람객별점, 관람객리뷰])

# # 평론가 리뷰
# lis2 = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')

# for li2 in lis2:
#     평론가별점 = li2.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
#     평론가리뷰 = li2.select_one('div.area_text_expand').text.strip()
#     df3.append([movie_name, 제목, 평론가별점, 평론가리뷰])

# # 기본정보 클릭 러닝타임 추출
# driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
# time.sleep(2)


# sub_soup = BeautifulSoup(driver.page_source, "html.parser")

# divs = sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')

# 러닝타임 = '-'

# for div in divs:
#     if '러닝타임' in div.text:
#         러닝타임 = div.text.split()[1]

# df1.append([movie_name, 제목, 러닝타임, 전체, 참여자수, 남자, 여자, _10대, _20대, _30대, _40대, _50대])  

In [68]:
star.to_excel('[크롤링] 역대_네이버_영화_평점_원본.xlsx', index=False)

In [69]:
review.to_excel('[크롤링] 역대_네이버_영화_관람객_리뷰_원본.xlsx', index=False)

In [71]:
critic_review.to_excel('[크롤링] 역대_네이버_영화_전문가_리뷰_원본.xlsx', index=False)

In [72]:
star.to_csv('[크롤링] 역대_네이버_영화_평점_원본.csv', index=False)

In [73]:
review.to_csv('[크롤링] 역대_네이버_영화_관람객_리뷰_원본.csv', index=False)

In [74]:
critic_review.to_csv('[크롤링] 역대_네이버_영화_전문가_리뷰_원본.csv', index=False)

# 코드 수정

In [288]:
best_movie = pd.read_excel("[KOBIS] 역대_박스오피스(2023.05.11).xlsx", header=4, engine="openpyxl")

df1 = pd.DataFrame(columns = ['순위', '영화명', '개봉일', '매출액', '관객수', '스크린수', '상영횟수', '대표국적', '국적', '배급사', 
                              '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
df7 = pd.DataFrame(columns = ['구분','별점','리뷰','제목','개봉','영화명','개봉일'])

fail_list=[]

# 셀레니움으로 네이버 창 열기
driver = webdriver.Chrome()
time.sleep(2)
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94&oquery=%EC%98%81%ED%99%94+%EC%95%84%EB%B0%94%ED%83%80&tqi=iaX7Llp0J1ZssSx9h9ZssssstjZ-258685"
driver.get(url)
time.sleep(3)

for i in tqdm(range(0,len(best_movie['영화명']))):
        
    info = best_movie.iloc[i]
    info_df = best_movie.iloc[i:i+1]
    
    try: 
        # 검색어 설정
        keyword = "영화 "+ str(info['영화명']) +" 평점"

        # 검색어 입력 및 검색
        driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').clear()
        driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').send_keys(keyword)
        driver.find_element(By.CSS_SELECTOR,'button.bt_search').click()
        time.sleep(3)

        # 리뷰 공감순 선택
        driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
        time.sleep(2)


        # 관람객 리뷰 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
        time.sleep(1)

        for j in range(0,2):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)

        # 평론가 클릭 및 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
        driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()
        time.sleep(1)

        for k in range(0,2):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        제목 = "-"
        전체 = "-"
        참여자수 = "-"
        남자 = "-"
        여자 = "-"

        _10대 = "-"
        _20대 = "-"
        _30대 = "-"
        _40대 = "-"
        _50대 = "-"

        
        제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
        전체 = soup.select_one('span.area_star_number').text
        참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
        남자 = soup.select_one('div.area_card_male span.area_star_number').text
        여자 = soup.select_one('div.area_card_female span.area_star_number').text

        _10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
        _20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
        _30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
        _40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
        _50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text

        df4 = []
        df5 = []

        # 관람객 리뷰
        lis1 = soup.select('div.lego_review_list._scroller ul._item_wrapper li')
        
        관람객별점 = '-'
        관람객리뷰 = '-'
        
        for li1 in lis1:
            관람객별점 = li1.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            관람객리뷰 = li1.select_one('div.area_text_expand span.desc._text').text.strip()
            df4.append(['관람객', 관람객별점, 관람객리뷰])

        # 평론가 리뷰
        lis2 = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')
        
        평론가별점 = '-'
        평론가리뷰 = '-'
        
        for li2 in lis2:
            평론가별점 = li2.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            평론가리뷰 = li2.select_one('div.area_text_expand').text.strip()
            df5.append(['평론가', 평론가별점, 평론가리뷰])

        # 기본정보 클릭 러닝타임 추출
        driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
        time.sleep(3)

        sub_soup = BeautifulSoup(driver.page_source, "html.parser")
    
        divs = sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')
        
        러닝타임 = '-'
        개봉 = '-'

        for div in divs:
            if '러닝타임' in div.text:
                러닝타임 = div.text.split()[1]
            elif '개봉' in div.text:
                개봉 = datetime.strptime(div.text.split()[1], '%Y.%m.%d.')

        
        df2_list = [[제목, 러닝타임, 개봉, 전체, 참여자수, 남자, 여자, _10대, _20대, _30대, _40대, _50대]]
        df2 = pd.DataFrame(df2_list, columns = ['제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'], index=[i])
        df3 = pd.concat([info_df, df2], axis=1)
        
        df1 = pd.concat([df1, df3])
        
        df4 = pd.DataFrame(df4, columns = ['구분','별점','리뷰'])
        df4['제목'] = 제목
        df4['개봉'] = 개봉
        df4['영화명'] = info['영화명']
        df4['개봉일'] = info['개봉일']

        df5 = pd.DataFrame(df5, columns = ['구분','별점','리뷰'])
        df5['제목'] = 제목
        df5['개봉'] = 개봉
        df5['영화명'] = info['영화명']
        df5['개봉일'] = info['개봉일']
         
        df6 = pd.concat([df4,df5])
        df7 = pd.concat([df7,df6])
        
    except:
        print(info['영화명'])
        fail_list.append([info['영화명'],info['개봉일']])
        
best_movie_star = pd.DataFrame(df1, columns=['순위', '영화명', '개봉일', '매출액', '관객수', '스크린수', '상영횟수', '대표국적', '국적', '배급사', 
                                             '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
best_movie_review = pd.DataFrame(df7, columns=['구분','별점','리뷰','제목','개봉','영화명','개봉일'])
best_movie_review = best_movie_review[['영화명','개봉일', '제목','개봉', '구분','별점','리뷰']]
fail_list_df = pd.DataFrame(fail_list, columns=['영화명','개봉일'])

best_movie_star.to_excel('[크롤링] 역대_네이버_영화_평점_수정.xlsx', index=False)
best_movie_review.to_excel('[크롤링] 역대_네이버_영화_리뷰_수정.xlsx', index=False)
fail_list_df.to_csv('[크롤링] 역대_네이버_영화_실패_수정.csv', index=False)

In [286]:
best_movie_review[(best_movie_review['영화명']=='극한직업')&(best_movie_review['구분']=='평론가')]['리뷰'][0]

'해체 위기에 놓인 마약 수사반이 잠복근무를 위해 치킨집을 인수한다. 공무원이 자영업자가 되면... 펼쳐보기'

In [162]:
best_movie_star

,순위,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,국적,배급사,...,개봉,전체,참여자수,남자,여자,_10대,_20대,_30대,_40대,_50대
0,1,명량,2014-07-30,135758658810,17615919,1587,188726,한국,한국,(주)씨제이이엔엠,...,2014-07-30,8.88,"28,765",8.82,8.94,9.18,8.92,8.76,8.84,8.82
1,2,극한직업,2019-01-23,139655543516,16266338,2003,292816,한국,한국,(주)씨제이이엔엠,...,2019-01-23,9.20,"15,943",9.16,9.24,9.43,9.32,9.22,9.09,8.82
2,3,신과함께-죄와 벌,2017-12-20,115727528087,14414658,1912,214631,한국,한국,롯데쇼핑㈜롯데엔터테인먼트,...,2017-12-20,8.73,"26,216",8.71,8.75,9.09,8.76,8.66,8.73,8.44
3,4,국제시장,2014-12-17,110947621230,14264478,1044,212705,한국,한국,(주)씨제이이엔엠,...,2014-12-17,9.16,"20,296",9.21,9.11,9.41,9.20,9.09,9.11,9.14
4,5,어벤져스: 엔드게임,2019-04-24,122492181020,13977602,2835,246433,미국,미국,월트디즈니컴퍼니코리아 유한책임회사,...,2019-04-24,9.50,"31,975",9.52,9.45,9.81,9.65,9.48,9.16,8.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,해리 포터와 불의 잔,2005-12-01,22097731900,3499257,504,45555,미국,미국,워너브러더스 코리아(주),...,2005-12-01,9.41,128,9.00,9.50,9.58,9.39,9.38,9.24,9.75
196,197,비긴 어게인,2014-08-13,27433947031,3479444,525,87565,미국,미국,판씨네마(주),...,2014-08-13,9.13,"7,381",9.18,9.09,9.41,9.22,8.98,8.79,8.89
197,198,태풍,2005-12-14,22209675300,3471150,481,47155,한국,한국,CJ ENM,...,2005-12-14,6.13,"6,934",5.63,6.98,6.52,5.97,5.95,6.41,5.25
198,199,킹콩,2005-12-14,21476688172,3470059,319,38456,미국,미국,UIP코리아,...,2005-12-14,8.97,"9,200",9.01,8.91,8.91,9.05,8.84,8.99,9.24


In [160]:
fail_list_df

,영화명,개봉일
0,겨울왕국,2014-01-16
1,엑시트,2019-07-31
2,디워,2007-08-01
3,트랜스포머,2007-06-28
4,캡틴 마블,2019-03-06
5,캐리비안의 해적: 세상의 끝에서,2007-05-23
6,가문의 위기(가문의 영광2),2005-09-07
7,아이언맨,2008-04-30
8,박물관이 살아있다!,2006-12-21
9,인디아나 존스 4 : 크리스탈 해골의 왕국,2008-05-22


In [163]:
best_movie_star.to_csv('[크롤링] 역대_네이버_영화_평점_수정.csv', index=False)
best_movie_review.to_csv('[크롤링] 역대_네이버_영화_리뷰_수정.csv', index=False)